In [5]:
import os
from openai import OpenAI

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model='gpt-4o-mini-2024-07-18',
    messages=[{'role':'user', 'content':'gpt-4o와 gpt-4o-mini-2024-07-18의 차이에 대해 알려줘'}],
    temperature=0.0
)

print(completion.choices[0].message.content)


GPT-4o와 GPT-4o-mini-2024-07-18은 OpenAI의 두 가지 모델로, 주로 크기와 성능, 용도에 따라 차이가 있습니다. 

1. **모델 크기**:
   - **GPT-4o**: 일반적으로 더 큰 모델로, 더 많은 파라미터를 가지고 있어 복잡한 작업을 수행하는 데 유리합니다.
   - **GPT-4o-mini**: 상대적으로 작은 모델로, 경량화되어 있어 빠른 응답 속도와 낮은 자원 소모가 특징입니다.

2. **성능**:
   - **GPT-4o**: 더 높은 성능을 제공하며, 복잡한 질문이나 작업에 대해 더 깊이 있는 답변을 생성할 수 있습니다.
   - **GPT-4o-mini**: 성능은 다소 떨어질 수 있지만, 여전히 많은 작업을 수행할 수 있으며, 특히 자원이 제한된 환경에서 유용합니다.

3. **용도**:
   - **GPT-4o**: 연구, 고급 애플리케이션, 복잡한 대화 시스템 등에서 사용됩니다.
   - **GPT-4o-mini**: 모바일 애플리케이션, 실시간 응답이 필요한 서비스 등에서 더 적합합니다.

4. **업데이트 날짜**:
   - **GPT-4o-mini-2024-07-18**: 이 모델은 특정 날짜에 업데이트된 버전으로, 최신 데이터와 개선 사항이 반영되어 있을 가능성이 높습니다.

이러한 차이점들은 사용자의 필요에 따라 적합한 모델을 선택하는 데 도움이 됩니다.


In [8]:
import json

with open('./resource/reviews.json', 'r') as f:
    reviews = json.load(f)

reviews[:3]

[{'review': '적절한 가격에 9시간 스테이를 활용할 수 있어서 좋았습니다.', 'stars': 5, 'date': '1일 전'},
 {'review': '시간대 선택이 좋으네요', 'stars': 4, 'date': '1일 전'},
 {'review': '지하철 역과 가깝고 코엑스 및 주변 부대시설과 접근이 용이해요~',
  'stars': 5,
  'date': '2024.09.04'}]

In [16]:
# good = 별점 5개
# bad = 별점 4개 이하
SPECIAL_TOKEN_PREFIX = '[REVIEW_START]'
SPECIAL_TOKEN_SUFFIX = '[REVIEW_END]'
good_reviews, bad_reviews = [], []
for r in reviews:
    if r['stars'] >= 5:
        good_reviews.append(f"{SPECIAL_TOKEN_PREFIX}{r['review']}{SPECIAL_TOKEN_SUFFIX}")
    else:
        bad_reviews.append(f"{SPECIAL_TOKEN_PREFIX}{r['review']}{SPECIAL_TOKEN_SUFFIX}")

print(good_reviews[:3])
print(bad_reviews[:3])
print(len(good_reviews), len(bad_reviews))

good_reviews_text = '\n'.join(good_reviews)
bad_reviews_text = '\n'.join(bad_reviews)

bad_reviews_text[:100]

['[REVIEW_START]적절한 가격에 9시간 스테이를 활용할 수 있어서 좋았습니다.[REVIEW_END]', '[REVIEW_START]지하철 역과 가깝고 코엑스 및 주변 부대시설과 접근이 용이해요~[REVIEW_END]', '[REVIEW_START]깨끗하고 부대시설이 좋았어요!![REVIEW_END]']
['[REVIEW_START]시간대 선택이 좋으네요[REVIEW_END]', '[REVIEW_START]진짜 작아요 정말로 그냥접근성 하나만보고 잤어요[REVIEW_END]', '[REVIEW_START]네 그냥 전반적으로 그냥 그래요[REVIEW_END]']
165 55


'[REVIEW_START]시간대 선택이 좋으네요[REVIEW_END]\n[REVIEW_START]진짜 작아요 정말로 그냥접근성 하나만보고 잤어요[REVIEW_END]\n[REVIEW_'

In [ ]:
# 전처리 함수
def preprocess_reviews():
    pass